In [1]:
from abstractions.goap.spatial import GridMap, ActionsPayload, ActionInstance, Attribute, BlocksMovement, BlocksLight, Node, GameEntity
from abstractions.goap.interactions import Character, Door, Key, Treasure, MoveStep, PickupAction, DropAction, UnlockAction, OpenAction, CloseAction, HealAction, Floor, IsPickupable, InanimateEntity
from pydantic import ValidationError



In [2]:


def visualize_grid(grid_map: GridMap):
    for y in range(grid_map.height):
        row = ""
        for x in range(grid_map.width):
            node = grid_map.get_node((x, y))
            if any(isinstance(entity, Character) for entity in node.entities):
                row += "C "
            elif any(isinstance(entity, Door) for entity in node.entities):
                row += "D "
            elif any(isinstance(entity, Key) for entity in node.entities):
                row += "K "
            elif any(isinstance(entity, Treasure) for entity in node.entities):
                row += "T "
            elif any(isinstance(entity, InanimateEntity) and entity.blocks_movement.value for entity in node.entities):
                row += "# "
            else:
                row += ". "
        print(row)
    print()

def generate_dungeon(grid_map: GridMap, room_width: int, room_height: int):
    room_x = (grid_map.width - room_width) // 2
    room_y = (grid_map.height - room_height) // 2

    for x in range(room_x, room_x + room_width):
        for y in range(room_y, room_y + room_height):
            if x == room_x or x == room_x + room_width - 1 or y == room_y or y == room_y + room_height - 1:
                if (x, y) != (room_x + room_width // 2, room_y):
                    wall = InanimateEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
                    grid_map.get_node((x, y)).add_entity(wall)
            else:
                floor = Floor(name=f"Floor_{x}_{y}")
                grid_map.get_node((x, y)).add_entity(floor)

    door_x, door_y = room_x + room_width // 2, room_y
    character_x, character_y = room_x + room_width // 2, room_y - 1
    key_x, key_y = room_x - 1, room_y + room_height // 2
    treasure_x, treasure_y = room_x + room_width // 2, room_y + room_height - 2

    door = Door(name="Door", is_locked=Attribute(name="is_locked", value=True), required_key=Attribute(name="required_key", value="Golden Key"))
    character = Character(name="Player")
    key = Key(name="Golden Key", key_name=Attribute(name="key_name", value="Golden Key"), is_pickupable=IsPickupable(value=True))
    treasure = Treasure(name="Treasure", monetary_value=Attribute(name="monetary_value", value=1000))

    grid_map.get_node((door_x, door_y)).add_entity(door)
    grid_map.get_node((character_x, character_y)).add_entity(character)
    grid_map.get_node((key_x, key_y)).add_entity(key)
    grid_map.get_node((treasure_x, treasure_y)).add_entity(treasure)

    for x in range(grid_map.width):
        for y in range(grid_map.height):
            if not any(isinstance(entity, Floor) for entity in grid_map.get_node((x, y)).entities):
                floor = Floor(name=f"Floor_{x}_{y}")
                grid_map.get_node((x, y)).add_entity(floor)

    return character, door, key, treasure

grid_map = GridMap(width=10, height=10)
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)

print("Initial grid state:")
visualize_grid(grid_map)

# Test neighborhood checks
move_action = MoveStep()
floor_entity = next(entity for entity in character.node.neighbors()[1].entities if isinstance(entity, Floor) and not entity.node.blocks_movement)
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=floor_entity.id, action=move_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Move action to neighboring node should succeed"

print("After moving to a neighboring node:")
visualize_grid(grid_map)

floor_entity = next(entity for entity in grid_map.get_node((0, 0)).entities if isinstance(entity, Floor))
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=floor_entity.id, action=move_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert not result.results[0].success, "Move action to non-neighboring node should fail"

print("Neighborhood checks passed!")

# Test inventory consequences
key_node = key.node
path = grid_map.a_star(character.node, key_node)
assert path is not None, "There should be a valid path from the character to the key"

for node in path.nodes[1:]:
    floor_entity = next(entity for entity in node.entities if isinstance(entity, Floor))
    move_action = MoveStep()
    actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=floor_entity.id, action=move_action)])
    result = grid_map.apply_actions_payload(actions_payload)
    assert result.results[0].success, f"Move action to node {node.position.value} should succeed"

print("After moving to the key's node:")
visualize_grid(grid_map)

assert character.node == key_node, "Character should be in the same node as the key"

pickup_action = PickupAction()
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=key.id, action=pickup_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Pickup action should succeed"
assert key in character.inventory, "Key should be in the character's inventory"
assert key not in key_node.entities, "Key should be removed from its original node"

print("After picking up the key:")
visualize_grid(grid_map)

print("Inventory consequences passed!")

# Move the character to the node next to the door
door_neighbor_node = grid_map.get_node((door.node.position.value[0]+1, door.node.position.value[1] + 1))
door_neighbor_node = grid_map.get_node((5,1))

path = grid_map.a_star(character.node, door_neighbor_node)
assert path is not None, "There should be a valid path from the character to the door's neighboring node"

for node in path.nodes[1:]:
    floor_entity = next(entity for entity in node.entities if isinstance(entity, Floor))
    move_action = MoveStep()
    actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=floor_entity.id, action=move_action)])
    result = grid_map.apply_actions_payload(actions_payload)
    assert result.results[0].success, f"Move action to node {node.position.value} should succeed"

print("After moving to the door's neighboring node:")
visualize_grid(grid_map)

assert character.node == door_neighbor_node, "Character should be at the door's neighboring node"

# Test door functionality
try:
    raycast_result = grid_map.get_raycast(character.node, treasure.node)
    assert not raycast_result.has_path, "Raycasting should fail due to the locked door"
except ValidationError as e:
    print(f"Raycasting failed as expected: {str(e)}")

path = grid_map.a_star(character.node, treasure.node)
assert path is None, "Pathfinding should fail due to the locked door"

unlock_action = UnlockAction()
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=door.id, action=unlock_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Unlock action should succeed"
assert not door.is_locked.value, "Door should be unlocked"

open_action = OpenAction()
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=door.id, action=open_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Open action should succeed"
assert door.open.value, "Door should be open"
print(door)
print(Door.get_instance(door.id))

assert not door.blocks_movement.value, "Door should not block movement"
assert not door.blocks_light.value, "Door should not block light"

print("After unlocking and opening the door:")
visualize_grid(grid_map)
print(Door.get_instance(door.id))
print(character.node.position, treasure.node.position)
raycast_result = grid_map.get_raycast(character.node, treasure.node)
# assert raycast_result.has_path, "Raycasting should succeed through the open door"

path = grid_map.a_star(character.node, treasure.node)
assert path is not None, "Pathfinding should succeed through the open door"

close_action = CloseAction()
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=door.id, action=close_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Close action should succeed"
assert not door.open.value, "Door should be closed"

print("After closing the door:")
visualize_grid(grid_map)

raycast_result = grid_map.get_raycast(character.node, treasure.node)
assert not raycast_result.has_path, "Raycasting should fail due to the closed door"

path = grid_map.a_star(character.node, treasure.node)
assert path is None, "Pathfinding should fail due to the closed door"

print("Door functionality passed!")

# Test the HealAction
character.health.value = 50
heal_action = HealAction()
actions_payload = ActionsPayload(actions=[ActionInstance(source_id=character.id, target_id=character.id, action=heal_action)])
result = grid_map.apply_actions_payload(actions_payload)
assert result.results[0].success, "Heal action should succeed"
assert character.health.value == min(50 + character.attack_power.value, character.max_health.value), "Character's health should be increased by the attack power"

print("HealAction passed!")
print("All tests passed!")

Updating block attributes... for door
Initial grid state:
. . . . . . . . . . 
. . . . . C . . . . 
. . # # # D # # . . 
. . # . . . . # . . 
. . # . . . . # . . 
. K # . . . . # . . 
. . # . . T . # . . 
. . # # # # # # . . 
. . . . . . . . . . 
. . . . . . . . . . 

Applying 1 actions
After moving to a neighboring node:
. . . . . C . . . . 
. . . . . . . . . . 
. . # # # D # # . . 
. . # . . . . # . . 
. . # . . . . # . . 
. K # . . . . # . . 
. . # . . T . # . . 
. . # # # # # # . . 
. . . . . . . . . . 
. . . . . . . . . . 

Applying 1 actions
Neighborhood checks passed!
Applying 1 actions
Applying 1 actions
Applying 1 actions
Applying 1 actions
Applying 1 actions
Applying 1 actions
Applying 1 actions
After moving to the key's node:
. . . . . . . . . . 
. . . . . . . . . . 
. . # # # D # # . . 
. . # . . . . # . . 
. . # . . . . # . . 
. C # . . . . # . . 
. . # . . T . # . . 
. . # # # # # # . . 
. . . . . . . . . . 
. . . . . . . . . . 

Applying 1 actions
After picking up the ke